In [1]:
import glob, h5py, os, string
import numpy as np
import matplotlib.pyplot as plt

from oc_ica import gabor_fit, plotting, analysis

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '6066' (I am process '9173')
Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


In [2]:
folder = '/home/jesse/Development/results/oc_ica/nat_images_mixtures-64_sources-128/'
files = sorted(glob.glob(os.path.join(folder, '*.h5')))
models = [f.split('.')[0].split('-')[-1] for f in files]
n_mixtures = int(folder.split('-')[-2].split('_')[0])
n_sources = int(folder.split('-')[-1].split('_')[0][:-1])
print(models)

['2', '4', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SM']


In [3]:
ws = np.full((len(models), 50, n_sources, n_mixtures), np.nan)
for ii, fname in enumerate(files):
    with h5py.File(fname) as f:
        print f['W_fits'].shape
        print f.keys()
        print f['sparsities'].value
        print f['lambdas'].value
        ws[ii] = np.squeeze(f['W_fits'].value)

(1, 50, 128, 64)
[u'W_fits', u'lambdas', u'sparsities']
[[ 38.00100327  37.99792099  38.00440216  37.99276733  37.99581528
   37.99588394  37.993927    37.99922562  37.9982338   37.99428558
   38.00640869  37.99622726  38.00567245  37.99065018  37.99565506
   37.99410248  38.00009537  37.99168777  38.00177383  37.99485016
   37.99470901  37.99734497  37.99440765  38.00345612  37.99760056
   37.99532318  37.99415588  37.99459457  38.00813293  37.99678802
   37.99217224  37.99322891  37.99886703  37.99053192  37.9943924
   37.9992981   37.99899292  38.00030518  38.00253296  37.99785995
   38.00358963  38.00434875  38.0004921   38.0009613   37.99484253
   38.00646973  37.99070358  38.00048828  38.00904846  37.99990082]]
[[ 64.64677659  66.11690262  66.11690262  66.11690262  66.11690262
   66.11690262  66.11690262  66.11690262  66.11690262  66.11690262
   64.64677659  66.11690262  66.11690262  66.11690262  66.11690262
   65.37770746  66.11690262  66.11690262  64.64677659  66.11690262
   66

In [6]:
params = plotting.get_Gabor_params(ws[:, 0])

In [17]:
ws_null.shape

(7, 128, 64)

In [24]:
rng = np.random.RandomState(2017)
ws_null = ws[:, 0].copy()

for ii in range(ws_null.shape[0]):
    for jj in range(ws_null.shape[2]):
        order = rng.permutation(ws_null.shape[1])
        ws_null[ii, :, jj] = ws_null[ii, :, jj][order]
    ws_null[ii] = ws_null[ii]/np.linalg.norm(ws_null[ii], axis=1, keepdims=True)
params_null = plotting.get_Gabor_params(ws_null)

In [88]:
n_mixtures = 64

In [132]:
reload(plotting)
n = 0
idxs = [0, 1, 3]
modelsp = [models[ii] for ii in idxs]
f = plt.figure(figsize=(2*len(modelsp), 6))
for ii, m in zip(idxs, modelsp):
    labelx = True
    if n//3 == 0:
        labely = True
    else:
        labely = False
    pos = (3, len(modelsp), n//len(modelsp)+1)
    plotting.plot_GaborFit_xy(params[ii][1], int(np.sqrt(n_mixtures)), m, f=f, pos=pos,
                              labelx=labelx, labely=labely)
    print pos
    n += 1
    pos = (3, len(modelsp), n//len(modelsp)+4)
    plotting.plot_GaborFit_polar(params[ii][1], m, f=f, pos=pos,
                                 labelx=labelx, labely=False)
    print pos
    n += 1
    pos = (3, len(modelsp), n//len(modelsp)+7)
    plotting.plot_GaborFit_envelope(params[ii][1], m, f=f, pos=pos,
                                    labelx=labelx, labely=labely)
    print pos
    n += 1
f.text(.09, .92, 'a)', fontsize=18)
f.text(.09, .58, 'b)', fontsize=18)
f.text(.09, .35, 'c)', fontsize=18)
f.text(.055, .57, 'planar rotation', rotation='vertical')
#plt.tight_layout()
plt.savefig('/home/jesse/Downloads/figure5.pdf')
#plt.show()
plt.close()

(3, 3, 1)
(3, 3, 4)
(3, 3, 7)
(3, 3, 2)
(3, 3, 5)
(3, 3, 8)
(3, 3, 3)
(3, 3, 6)
(3, 3, 9)


In [26]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.hist(np.sqrt(1./params[0][2]))
ax1.set_title('data')

ax2.hist(np.sqrt(1./params_null[0][2]))
ax2.set_title('null')
plt.show()

In [31]:
fitter = gabor_fit.GaborFit()

In [28]:
n = 0

In [29]:
reload(plotting)
plotting.plot_bases(ws[n, 0])
wp = fitter.make_gabor(params[n][0][-1], 8, 8).T
wp = wp.reshape(wp.shape[0], -1)
plotting.plot_bases(wp)
plt.show()

NameError: name 'fitter' is not defined

In [34]:
reload(plotting)
plotting.plot_bases(ws_null[n])
wp = fitter.make_gabor(params_null[n][0][-1], 8, 8).T
wp = wp.reshape(wp.shape[0], -1)
plotting.plot_bases(wp)
plt.show()

In [26]:
reload(plotting)
plotting.plot_figure1c('/home/jesse/Downloads/figure1c.pdf')

(<matplotlib.figure.Figure at 0x7f7380c4dc50>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f7380987650>)

In [19]:
wsp = ws.copy()
ws = ws[:, [0], ...]

In [23]:
n = -1
print ws[n, 0]
print ''
print ws[n, 2]

[[ 0.00093331 -0.00871581 -0.01035803 ..., -0.00476439  0.00230579
   0.01385963]
 [-0.00261299  0.00717153  0.01643513 ..., -0.40902462  0.18723013
   0.00909613]
 [-0.00169378 -0.01118835  0.00483536 ...,  0.01423424 -0.01329052
  -0.00273621]
 ..., 
 [ 0.21734912  0.37128276  0.45883086 ..., -0.001153    0.00173251
   0.01015912]
 [ 0.00046596 -0.04129926 -0.06906662 ...,  0.06579425 -0.00905298
   0.00386784]
 [-0.00674613 -0.00801815  0.00530376 ..., -0.04284265  0.05979943
   0.04687073]]

[[-0.01217862  0.00536091  0.16521776 ...,  0.00232122 -0.01246881
   0.01375526]
 [-0.00370093  0.00607227 -0.00184398 ..., -0.19924941  0.25794092
   0.51573407]
 [ 0.00385826  0.00441612 -0.00399309 ..., -0.07832332 -0.42166133
   0.26441595]
 ..., 
 [ 0.00165046 -0.02065036  0.03471835 ..., -0.01007877  0.01139323
   0.02096904]
 [ 0.01321779 -0.00298589  0.00240167 ..., -0.59966038  0.5169174
  -0.1104824 ]
 [-0.02145762  0.02089402  0.0084813  ...,  0.01215959 -0.00091219
   0.00472633]]


In [10]:
reload(plotting)
idxs1 = [6, 0, 1, 4, 2]
idxs2 = [1, 4, 5, 2, 3]
plotting.plot_figure3(ws[idxs1], [models[ii] for ii in idxs1], 1, ws[idxs2], [models[ii] for ii in idxs2],
                      save_path='/home/jesse/Downloads/figure4.pdf')

In [13]:
reload(plotting)
plotting.plot_figure1c('/home/jesse/Downloads/figure1c.pdf')

(<matplotlib.figure.Figure at 0x7f4ebceff690>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f4ebce4ca90>)

In [56]:
reload(plotting)
plotting.plot_figure2('/home/jesse/Downloads/figure2.pdf', n_iter=1)
plt.close()

Overcomplete ICA
Degeneracy control: L2
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.9999923706
Overcomplete ICA
Degeneracy control: COHERENCE_SOFT
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.0680210739374
Overcomplete ICA
Degeneracy control: COULOMB
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.4725198531
Overcomplete ICA
Degeneracy control: RANDOM
setup
transform
ICA with L-BFGS-B done!
Final loss value: 127.292408908
Overcomplete ICA
Degeneracy control: L4
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.275804817677
Overcomplete ICA
Degeneracy control: COHERENCE_SOFT
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.224166229367
Overcomplete ICA
Degeneracy control: RANDOM
setup
transform
ICA with L-BFGS-B done!
Final loss value: 127.293999051
Overcomplete ICA
Degeneracy control: COULOMB
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.4696479821
Overcomplete ICA
Degeneracy control: L2
setup
transform
IC

In [46]:
reload(plotting)
plotting.plot_figure2a(save_path='/home/jesse/Downloads/figure2a.pdf', n_iter=10)
plotting.plot_figure2b(save_path='/home/jesse/Downloads/figure2b.pdf', n_iter=10)
plotting.plot_figure2c(save_path='/home/jesse/Downloads/figure2c.pdf', n_iter=10)
plotting.plot_figure2de('d', save_path='/home/jesse/Downloads/figure2d.pdf')
plotting.plot_figure2de('e', save_path='/home/jesse/Downloads/figure2e.pdf')
plt.close()

Overcomplete ICA
Degeneracy control: L2
setup
transform
ICA with L-BFGS-B done!
Final loss value: 64.0000076294
Overcomplete ICA
Degeneracy control: COHERENCE_SOFT
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.0690286904573
Overcomplete ICA
Degeneracy control: COULOMB
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.4743571066
Overcomplete ICA
Degeneracy control: RANDOM
setup
transform
ICA with L-BFGS-B done!
Final loss value: 127.293631178
Overcomplete ICA
Degeneracy control: L4
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.276505202055
Overcomplete ICA
Degeneracy control: L2
setup
transform
ICA with L-BFGS-B done!
Final loss value: 64.0000762939
Overcomplete ICA
Degeneracy control: COHERENCE_SOFT
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.0
Overcomplete ICA
Degeneracy control: COULOMB
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.4717067265
Overcomplete ICA
Degeneracy control: RANDOM
setup
transform
ICA with L-BF

/home/jesse/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Overcomplete ICA
Degeneracy control: L2
setup
transform
ICA with L-BFGS-B done!
Final loss value: 64.0
Overcomplete ICA
Degeneracy control: COHERENCE_SOFT
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.292778283358
Overcomplete ICA
Degeneracy control: COULOMB
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.4708024048
Overcomplete ICA
Degeneracy control: RANDOM
setup
transform
ICA with L-BFGS-B done!
Final loss value: 127.295162624
Overcomplete ICA
Degeneracy control: L4
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.275657474995
Overcomplete ICA
Degeneracy control: L2
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.9999961853
Overcomplete ICA
Degeneracy control: COHERENCE_SOFT
setup
transform
ICA with L-BFGS-B done!
Final loss value: 0.175310283899
Overcomplete ICA
Degeneracy control: COULOMB
setup
transform
ICA with L-BFGS-B done!
Final loss value: 63.4695396208
Overcomplete ICA
Degeneracy control: RANDOM
setup
transform
ICA with L-B

# Score Matching

In [12]:
import os, h5py
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction import image

from oc_ica.datasets import zca
from oc_ica.utils import tile_raster_images as tri
from oc_ica.models import sc, ica
from oc_ica.plotting import plot_bases
from oc_ica.sparsity_search import sparsity_search
from oc_ica.optimizers.ica_optimizers import sgd, adam, momentum
reload(sc)
reload(ica)

<module 'oc_ica.models.ica' from '/home/jesse/Development/oc_ica/oc_ica/models/ica.pyc'>

In [13]:
filename = os.path.join(os.environ['SCRATCH'],'data/vanhateren/images_curated.h5')
key = 'van_hateren_good'

with h5py.File(filename,'r') as f:
    images = f[key].value

In [32]:
patch_size = 8
n_mixtures = patch_size**2
total_samples = 100000
rng = np.random.RandomState(20170125)
patches = image.PatchExtractor(patch_size=(patch_size, patch_size),
                               max_patches=total_samples//images.shape[0],
                               random_state=rng).transform(images)
X = patches.reshape((patches.shape[0],n_mixtures)).T
X_mean = X.mean(axis=-1, keepdims=True)
X -= X_mean
X_zca, d, u = zca(X)

In [33]:
OC = .75
n_sources = int(n_mixtures * float(OC))
model = ica.ICA(n_mixtures, n_sources, degeneracy='SM', lambd=0., rng=rng, norm_projection=True,
                #optimizer='sgd', learning_rule=adam, verbose=True)
                verbose=True)

Degeneracy control: SM
setup
transform


In [34]:
print(model.losses(X_zca))
model.fit(X_zca) #, n_epochs=1000)
print(model.losses(X_zca))

[array(-20.5378475189209, dtype=float32), array(-20.5378475189209, dtype=float32), array(0.0, dtype=float32), array(0.6628289222717285, dtype=float32)]
Loss: -23.120256424, Error: -23.120256424, Penalty: 0.0, MSE: 0.532486915588
Loss: -24.9580593109, Error: -24.9580593109, Penalty: 0.0, MSE: 0.423188567162
Loss: -25.6826133728, Error: -25.6826133728, Penalty: 0.0, MSE: 0.385157138109
Loss: -26.0141601562, Error: -26.0141601562, Penalty: 0.0, MSE: 0.368580043316
Loss: -26.2121334076, Error: -26.2121334076, Penalty: 0.0, MSE: 0.361532837152
Loss: -26.3592071533, Error: -26.3592071533, Penalty: 0.0, MSE: 0.367598444223
Loss: -26.5059890747, Error: -26.5059890747, Penalty: 0.0, MSE: 0.379490226507
Loss: -26.681842804, Error: -26.681842804, Penalty: 0.0, MSE: 0.407830774784
Loss: -26.8911457062, Error: -26.8911457062, Penalty: 0.0, MSE: 0.392673134804
Loss: -27.0122413635, Error: -27.0122413635, Penalty: 0.0, MSE: 0.392248541117
Loss: -27.240776062, Error: -27.240776062, Penalty: 0.0, MSE: 

In [35]:
sq = int(np.ceil(np.sqrt(n_sources)))
im = tri(model.components_, (patch_size, patch_size), (sq, sq), (2, 2))
plt.imshow(im, cmap='gray', interpolation='nearest')
plt.show()

# SM ICA

In [11]:
x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
x, y = np.meshgrid(x, y)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))

z = np.log(np.cosh(x)) + np.log(np.cosh(y))
z = z-z.min()
z = z/z.max()
ax1.contour(x, y, z, aspect='equal', cmap='viridis')
ax1.set_title('log(cosh(s))')
ax1.set_xlabel('s1')
ax1.set_ylabel('s2')

z = (-1+1.5*np.tanh(x)**2) + (-1+.5*np.tanh(y)**2)
z = z-z.min()
z = z/z.max()
ax2.contour(x, y, z, aspect='equal', cmap='viridis')
ax2.set_title('-1+.5tanh(s1)^2')
ax2.set_xlabel('s1')
ax2.set_ylabel('s2')

plt.tight_layout()
plt.show()